# Conflict Volatility (fatalities categories)


This notebook creates a way to observe countries' levels of violence and their likelihood of transitioning to various levels of violence over time periods. 

This notebook will create a new categorical variable which converts the actuals and predictions into the following groups: (0) Non-violent; (1) Low-violence; (2) Violent; (3) High-violence; (4) Extreme Violence.

**Non-violent** refers to zero conflict-related deaths in a given country for each month over the past 3-months.

**Low-violence** refers to 1-9 conflict-related deaths in a given country for each month over the past 3-months.

**Violent** refers to 9-99 or more conflict-related deaths in a given country for each month over the past 3-months.

**High-violence** refers to 100-1000 or more conflict-related deaths in a given country for each month over the past 3-months.

**Extreme violence** refers to 1000 or more conflict-related deaths in a given country for each month over the past 3-months.

### Configuration and Packages

In [1]:
# Basics
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.cbook as cbook
import warnings
warnings.filterwarnings('ignore')

# Views 3
from viewser.operations import fetch
from viewser import Queryset, Column
import views_runs
from views_partitioning import data_partitioner, legacy
from stepshift import views
from views_runs import storage, ModelMetadata
from views_runs.storage import store, retrieve, fetch_metadata
from views_forecasts.extensions import *
import views_mapper2
from views_mapper2.mapper2 import Mapper2
from views_mapper2 import color
from views_mapper2.label_writer import vid2date
from views_mapper2.dictionary_writer import standard_scale

# Packages from Mapper2
from ingester3.ViewsMonth import ViewsMonth
from ingester3.Country import Country
from datetime import date


# Packages from this repository, Tools folder
import sys
sys.path.append('../')
sys.path.append('../Tools')
sys.path.append('../Intermediates')
sys.path.append('../SystemUpdates')

import os
from pathlib import Path

In [2]:
dev_id = 'Fatalities002'
run_id = dev_id 
EndOfHistory = 509
prod_id = '2022_04_t01'
level = 'cm'
months_prior = 3

import os
home = os.path.expanduser("~")
Mydropbox = home + '/Dropbox (ViEWS)/ViEWS_new/'
Monthly_updates = Mydropbox + 'DataReleases/MonthlyUpdates/'
localgitpath = home + '/Desktop/VIEWS_new/'


##################
print('OS Login is:', os.getlogin())
print('OS path is set to:', home)
print('Local Git path is set to:', localgitpath)
print('Dropbox path is set to:', Mydropbox)

OS Login is: chandlerwilliams
OS path is set to: /Users/chandlerwilliams
Local Git path is set to: /Users/chandlerwilliams/Desktop/VIEWS_new/
Dropbox path is set to: /Users/chandlerwilliams/Dropbox (ViEWS)/ViEWS_new/


## 1: Fetching and merge the data


    1: Pull in the actuals, restrict the data to (EndOfHistory-40) to EndOfHistory

    2: Pull in the future categorical future predictions, steps 1 to 36 months into the future
    
    3: Merge the DFs: create a single df with actuals and predictions

### 1.1: Fetch acutals

The following cell calls fatalities per UCDP GED 

In [3]:
qs1 = (Queryset("actuals_monthly", "country_month")
            .with_column(Column("ged_best_sb", from_table= "ged2_cm", from_column="ged_sb_best_sum_nokgi")
                        .transform.missing.fill())   
            .with_theme("fatalities")
            .describe(""" Fatalities conflict history, cm level Monthly Update"""))
actuals = qs1.publish().fetch()

actuals = actuals.loc[EndOfHistory-40:EndOfHistory]
actuals = actuals.reset_index()
actuals['step'] = actuals['month_id'] - EndOfHistory
actuals = actuals.set_index(['step', 'country_id'], drop = True)
actuals.rename(columns={'ged_best_sb':'fatalities'}, inplace=True)

 .    

### 1.2: Fetch the future predictions with their categorical uncertainty values

Presently, this simply reads in the .csv file; however, I would like it to read from pred_store. It also transforms 'steps_combined' into the exponential form. 

In [4]:
future = pd.read_csv('Categorical_probabilities.csv')

future['step'] = future['month_id'] - EndOfHistory
future = future.set_index(['step', 'country_id'], drop = True)
future['step_combined'] = np.expm1(future['step_combined'])
future.rename(columns={'step_combined':'fatalities'}, inplace=True)

### 1.3: Merge the data


In [5]:
merge = pd.concat([actuals, future], axis=0)

In [ ]:
merge

## 2: Creating Fatalities Categories (Fat_Cat)

This section creates fat_cat. Here, I write the code the defines the categories. 

# 2.1: Defining the categories

This cell defines a function that creates a multiclass categorical variable grouping fatalities 
    
    0: Peaceful = a country-month without any fatalities
    1: Low-violence = a country-month with 1-9 fatalities
    2: Violent = a country-month with more than 9-99 fatalities
    3: High-violence = a country-month with more than 100-999 fatalities
    4: Extreme violence = a country-month with more than 1000 fatalites
    

In [6]:
fatcat = merge.copy()
fatcat = fatcat.reset_index(drop=False)

In [7]:
def ged_fatalities(x):
    if x < 0.5:
        return 0
    elif x < 10:
        return 1
    elif x < 100:
        return 2
    elif x < 1000:
        return 3
    else:
        return 4
       
def ged_intensity_lvl(x):
    if x < 0.5:
        return 0
    elif x < 100:
        return 1
    else:
        return 2  

In [8]:
fatcat['fatcat'] = fatcat['fatalities'].apply(ged_fatalities)

# Create a variable that subtracts the observation in the row above by the row below and prints the output
fatcat['fatcat_change'] = fatcat.groupby('country_id')['fatcat'].diff()


# Fill the top values of ['status'] with zeros
fatcat['fatcat_change'] = fatcat['fatcat_change'].fillna(0)

fatcat = fatcat.set_index(['step','country_id'], drop = True)


In [9]:
# This cell creates 'fatcat_nominal'

fatcat['fatcat_nominal'] = fatcat['fatcat']
fatcat = fatcat.reset_index()

fat_classes = [
    (fatcat['fatcat'] == 0), 
    (fatcat['fatcat'] == 1),
    (fatcat['fatcat'] == 2),
    (fatcat['fatcat'] == 3),
    (fatcat['fatcat'] == 4)
]

fat_categories = ['Non-violent', 'Low Violence', 'Violence', 'High-violence', 'Extreme Violence']

fatcat['fatcat_nominal'] = np.select(fat_classes, fat_categories)

fatcat = fatcat.set_index(['step','country_id'], drop = True)

In [ ]:
# This cell creates 'monthly_change' variable

#fatcat['monthly_change'] = fatcat['fatcat_change']
#fatcat = fatcat.reset_index()

#violence_conditions = [
#    (fatcat['monthly_change'] == -2), 
#    (fatcat['monthly_change'] == -1),
#    (fatcat['monthly_change'] == 0),
#    (fatcat['monthly_change'] == 1),
#    (fatcat['monthly_change'] == 2)
#]

#violence_categories = ['Rapid Descalation', 'De-escalation', 'Status Quo', 'Escalation', 'Rapid Escalation']

#fatcat['monthly_change'] = np.select(violence_conditions, violence_categories)

In [10]:
# This cell creates 'monthly_change' variable

fatcat['monthly_change'] = fatcat['fatcat_change']
fatcat = fatcat.reset_index()

violence_conditions = [
    (fatcat['monthly_change'] == -4),
    (fatcat['monthly_change'] == -3), 
    (fatcat['monthly_change'] == -2), 
    (fatcat['monthly_change'] == -1),
    (fatcat['monthly_change'] == 0),
    (fatcat['monthly_change'] == 1),
    (fatcat['monthly_change'] == 2),
    (fatcat['monthly_change'] == 3),
    (fatcat['monthly_change'] == 4)
]

violence_categories = ['Extreme Descalation', 'Rapid De-escalation', 'De-escalation', 
                       'Low De-escalation', 'Status Quo','Low Escalation',
                      'Escalation','Rapid Escalation','Extreme Escalation']

fatcat['monthly_change'] = np.select(violence_conditions, violence_categories)

In [11]:
#fatcat['monthly_change'] = np.select(violence_conditions, violence_categories)

# This creates a rolling mean of the violence over the past 'months prior' parameter
fatcat['rolling_mean_violence'] = fatcat.groupby('country_id')['fatalities'].transform(lambda x: x.shift(1).rolling(months_prior, 1).mean())

fatcat['intensity_lvl'] = fatcat['rolling_mean_violence'].apply(ged_intensity_lvl)

#Reset the df to only include 12 months prior
fatcat = fatcat.loc[fatcat['step'] >= -12]

#Reset the index
fatcat = fatcat.set_index(['step', 'country_id'], drop = True)


In [12]:
fatcat

month_id  fatalities  multi_0_logit  multi_1_logit  \
step country_id                                                       
-12  1                497    0.000000            NaN            NaN   
     2                497    0.000000            NaN            NaN   
     3                497    0.000000            NaN            NaN   
     4                497    1.000000            NaN            NaN   
     5                497    0.000000            NaN            NaN   
...                   ...         ...            ...            ...   
 36  242              545    2.981319       0.835207       0.091962   
     243              545    1.365385       0.922174       0.047673   
     244              545    0.234067       0.971128       0.019518   
     245              545    6.377980       0.645284       0.174019   
     246              545   12.277349       0.401805       0.254329   

                 multi_2_logit  multi_3_logit  multi_4_logit  fatcat  \
step country_id                                                        
-12  1                     NaN            NaN            NaN       0   
     2                     NaN            NaN            NaN       0   
     3                     NaN            NaN            NaN       0   
     4                     NaN            NaN            NaN       1   
     5                     NaN            NaN            NaN       0   
...                        ...            ...            ...     ...   
 36  242              0.069502       0.003328   8.502074e-07       1   
     243              0.029098       0.001055   1.421743e-07       1   
     244              0.009121       0.000234   1.415694e-08       0   
     245              0.169412       0.011279   6.147240e-06       1   
     246              0.315118       0.028715   3.220856e-05       2   

                 fatcat_change fatcat_nominal  monthly_change  \
step country_id                                                 
-12  1                     0.0    Non-violent      Status Quo   
     2                     0.0    Non-violent      Status Quo   
     3                     0.0    Non-violent      Status Quo   
     4                     1.0   Low Violence  Low Escalation   
     5                     0.0    Non-violent      Status Quo   
...                        ...            ...             ...   
 36  242                   0.0   Low Violence      Status Quo   
     243                   0.0   Low Violence      Status Quo   
     244                   0.0    Non-violent      Status Quo   
     245                   0.0   Low Violence      Status Quo   
     246                   0.0       Violence      Status Quo   

                 rolling_mean_violence  intensity_lvl  
step country_id                                        
-12  1                        0.000000              0  
     2                        0.000000              0  
     3                        0.000000              0  
     4                        0.000000              0  
     5                        0.000000              0  
...                                ...            ...  
 36  242                      2.587225              1  
     243                      1.414412              1  
     244                      0.274558              0  
     245                      7.290787              1  
     246                     16.659600              1  

[9359 rows x 13 columns]

In [ ]:
#fatcat.to_csv('/Users/chandlerwilliams/Desktop/VIEWS_new/test_folder/fatcatpeak.csv',sep = ';')

## 3: Creating Change Lists

The following cells shows the countries that are expected to shift between fatcats

### 3.1: Creating a list from Step0 

In [17]:
fatcat0 = fatcat.reset_index(drop=False)
fatcat0 = fatcat0[fatcat0['step'] == 0] 
fatcat0_pivot = pd.pivot_table(fatcat0, values=['fatalities', 'multi_0_logit','multi_1_logit', 'multi_2_logit', 'multi_3_logit',
                                                'multi_4_logit', 'fatcat', 'fatcat_nominal', 'fatcat_change', 'monthly_change', 'rolling_mean_violence',
                                                'intensity_lvl'],
                               index=['country_id'])

fatcat0_pivot.rename(columns={'fatalities': 'fatalities_step_0'}, inplace=True)
fatcat0_pivot.rename(columns={'fatcat': 'fatcat_step_0'}, inplace=True)

### 3.2: Creating a list from StepX 

In [18]:
fatcat12 = fatcat.reset_index(drop=False)
fatcat12 = fatcat12[fatcat12['step'] == 12] 
fatcat12_pivot = pd.pivot_table(fatcat12, values=['fatalities', 'multi_0_logit','multi_1_logit', 'multi_2_logit', 'multi_3_logit',
                                                'multi_4_logit', 'fatcat', 'fatcat_change', 'monthly_change', 'rolling_mean_violence',
                                                'intensity_lvl', 'fatcat_nominal'],
                               index=['country_id'])

fatcat12_pivot.rename(columns={'fatalities': 'fatalities_step_12'}, inplace=True)
fatcat12_pivot.rename(columns={'fatcat': 'fatcat_step_12'}, inplace=True)
#fatcat_3 = fatcat_3.sort_values(by=['fatalities'], ascending=False)

### 3.3: Joining these data 

In [19]:
fatcat_list = fatcat0_pivot.join(fatcat12_pivot, lsuffix='_step_0', rsuffix='_step_12')
fatcat_list = fatcat_list.sort_values('fatalities_step_0', ascending=False)

In [20]:
fatcat_list['fatcat_change'] = fatcat_list['fatcat_step_12'] - fatcat_list['fatcat_step_0']

violence_conditions2 = [
    (fatcat_list['fatcat_change'] == -4),
    (fatcat_list['fatcat_change'] == -3), 
    (fatcat_list['fatcat_change'] == -2), 
    (fatcat_list['fatcat_change'] == -1),
    (fatcat_list['fatcat_change'] == 0),
    (fatcat_list['fatcat_change'] == 1),
    (fatcat_list['fatcat_change'] == 2),
    (fatcat_list['fatcat_change'] == 3),
    (fatcat_list['fatcat_change'] == 4)
]



violence_categories = ['Extreme Descalation', 'Rapid De-escalation', 'De-escalation', 
                       'Low De-escalation', 'Status Quo','Low Escalation',
                      'Escalation','Rapid Escalation','Extreme Escalation']

fatcat_list['facat_change_nominal'] = np.select(violence_conditions2, violence_categories)

### 3.4: Data manipulation

In [21]:
def cid2name(country_id):
    """
    Function useful for labeling countries if needed by their name
    Create a string output name by country id
    Uses ingester3 Country as basis
    :param country_id: Views Country ID
    :return: name
    """
    name = Country(country_id).name
    return name

In [22]:
fatcat_list = fatcat_list.reset_index(drop=False)

In [23]:
c_id = [int(x) for x in fatcat_list['country_id']] 
country_names = []
for x in (c_id):
        country_names.append(cid2name(x)) 

In [24]:
fatcat_list['country'] = country_names
fatcat_list = fatcat_list.set_index('country_id', drop = True)

#### Creating 'fatcat_nominal' for steps 1 and 12

In [25]:
# Creates: Fatcat list 0 nominal

fatcat_list['fatcat_0_nominal'] = fatcat_list['fatcat_step_0']
fatcat_list = fatcat_list.reset_index()

fat_classes = [
    (fatcat_list['fatcat_step_0'] == 0), 
    (fatcat_list['fatcat_step_0'] == 1),
    (fatcat_list['fatcat_step_0'] == 2),
    (fatcat_list['fatcat_step_0'] == 3),
    (fatcat_list['fatcat_step_0'] == 4)
]

fat_categories = ['Non-violent', 'Low Violence', 'Violent', 'High-violence', 'Extreme Violence']

fatcat_list['fatcat_0_nominal'] = np.select(fat_classes, fat_categories)

fatcat_list = fatcat_list.set_index(['country_id'], drop = True)

In [26]:
# Creates: Fatcat list 12 nominal

fatcat_list['fatcat_12_nominal'] = fatcat_list['fatcat_step_12']
fatcat_list = fatcat_list.reset_index()

fat_classes = [
    (fatcat_list['fatcat_step_12'] == 0), 
    (fatcat_list['fatcat_step_12'] == 1),
    (fatcat_list['fatcat_step_12'] == 2),
    (fatcat_list['fatcat_step_12'] == 3),
    (fatcat_list['fatcat_step_12'] == 4)
]

fat_categories = ['Non-violent', 'Low Violence', 'Violent', 'High-violence', 'Extreme Violence']

fatcat_list['fatcat_12_nominal'] = np.select(fat_classes, fat_categories)

fatcat_list = fatcat_list.set_index(['country_id'], drop = True)

## 4: Fatcat Lists

### 4.1: Escalation List

This list the countries that are expected to esclate in violence from the EndOfHistory until 12-months later

In [27]:
escalationlist = fatcat_list.copy()
escalationlist = escalationlist.loc[escalationlist['fatcat_change'] >= 1 ] 
escalationlist = escalationlist.sort_values(by = ['fatalities_step_12'], ascending=[False])
escalationlist.to_markdown(buf= '/Users/chandlerwilliams/Desktop/VIEWS_new/test_folder/escalationlist_12.md')
escalationlist
#print(escalationlist.to_markdown())

,fatalities_step_0,fatcat_step_0,fatcat_change_step_0,intensity_lvl_step_0,rolling_mean_violence_step_0,fatalities_step_12,fatcat_step_12,fatcat_change_step_12,intensity_lvl_step_12,multi_0_logit,multi_1_logit,multi_2_logit,multi_3_logit,multi_4_logit,rolling_mean_violence_step_12,fatcat_change,facat_change_nominal,country,fatcat_0_nominal,fatcat_12_nominal
country_id,,,,,,,,,,,,,,,,,,,,
124,41.0,2,0.0,2,258.000000,581.768308,3,0.0,2,0.000017,0.028920,0.456718,0.441704,7.264129e-02,464.244780,1,Low Escalation,Yemen,Violent,High-violence
220,85.0,2,0.0,1,59.000000,233.068575,3,0.0,2,0.000210,0.069125,0.602248,0.304464,2.395246e-02,191.362203,1,Low Escalation,Syria,Violent,High-violence
223,38.0,2,0.0,1,30.333333,116.224225,3,0.0,1,0.001266,0.119706,0.664386,0.205403,9.239434e-03,99.473261,1,Low Escalation,India,Violent,High-violence
237,0.0,0,-1.0,1,9.666667,4.422479,1,0.0,1,0.604133,0.223135,0.166910,0.005800,2.174847e-05,5.858108,1,Low Escalation,Kenya,Non-violent,Low Violence
96,0.0,0,0.0,0,0.333333,3.655749,1,0.0,1,0.677290,0.190006,0.128679,0.004012,1.329996e-05,3.657932,1,Low Escalation,Turkey,Non-violent,Low Violence
246,0.0,0,0.0,0,0.333333,3.394828,1,0.0,1,0.702907,0.177706,0.115908,0.003469,1.097471e-05,3.762523,1,Low Escalation,South Sudan,Non-violent,Low Violence
214,0.0,0,-2.0,1,9.000000,3.236308,1,0.0,1,0.718556,0.170013,0.108265,0.003156,9.694537e-06,3.979258,1,Low Escalation,Chad,Non-violent,Low Violence
245,0.0,0,0.0,0,0.000000,2.464283,1,0.0,1,0.794383,0.130745,0.073022,0.001845,4.816462e-06,2.160593,1,Low Escalation,Sudan,Non-violent,Low Violence
128,0.0,0,0.0,0,0.000000,2.252714,1,0.0,1,0.814631,0.119669,0.064146,0.001550,3.844643e-06,2.725371,1,Low Escalation,Iran,Non-violent,Low Violence


### 4.1: De-escalation List

This list the countries that are expected to de-esclate in violence from the EndOfHistory until 12-months later

In [28]:
deescalationlist = fatcat_list.copy()
deescalationlist = deescalationlist.loc[deescalationlist['fatcat_change'] < 0 ] 
deescalationlist = deescalationlist.sort_values(by = ['fatalities_step_12'], ascending=[False])
deescalationlist.to_markdown(buf= '/Users/chandlerwilliams/Desktop/VIEWS_new/test_folder/deescalationlist_12.md')
#print(deescalationlist.to_markdown())
deescalationlist

,fatalities_step_0,fatcat_step_0,fatcat_change_step_0,intensity_lvl_step_0,rolling_mean_violence_step_0,fatalities_step_12,fatcat_step_12,fatcat_change_step_12,intensity_lvl_step_12,multi_0_logit,multi_1_logit,multi_2_logit,multi_3_logit,multi_4_logit,rolling_mean_violence_step_12,fatcat_change,facat_change_nominal,country,fatcat_0_nominal,fatcat_12_nominal
country_id,,,,,,,,,,,,,,,,,,,,
117,4405.0,4,0.0,2,6854.000000,81.222141,2,0.0,2,0.003078,0.153419,0.675692,0.162330,0.005482,128.723966,-2,De-escalation,Ukraine,Extreme Violence,Violent
78,117.0,3,0.0,1,64.000000,56.799116,2,0.0,1,0.007274,0.191981,0.671919,0.125636,0.003191,53.243377,-1,Low De-escalation,Niger,High-violence,Violent
137,46.0,2,2.0,0,0.333333,1.343148,1,0.0,1,0.895136,0.072766,0.031494,0.000603,0.000001,1.446490,-1,Low De-escalation,Tajikistan,Violent,Low Violence


### 4.1: Status Quo List

This list the countries that are expected to de-esclate in violence from the EndOfHistory until 12-months later

In [29]:
sqlist = fatcat_list.copy()
sqlist = sqlist.loc[sqlist['fatcat_change'] == 0 ] 
sqlist = sqlist.sort_values(by = ['fatalities_step_12'], ascending=[False])
sqlist.to_markdown(buf= '/Users/chandlerwilliams/Desktop/VIEWS_new/test_folder/sqlist_12.md')
#print(sqlist.to_markdown())
sqlist

,fatalities_step_0,fatcat_step_0,fatcat_change_step_0,intensity_lvl_step_0,rolling_mean_violence_step_0,fatalities_step_12,fatcat_step_12,fatcat_change_step_12,intensity_lvl_step_12,multi_0_logit,multi_1_logit,multi_2_logit,multi_3_logit,multi_4_logit,rolling_mean_violence_step_12,fatcat_change,facat_change_nominal,country,fatcat_0_nominal,fatcat_12_nominal
country_id,,,,,,,,,,,,,,,,,,,,
133,128.0,3,0.0,1,52.000000,619.633464,3,0.0,2,0.000014,0.027044,0.444993,0.450068,7.788086e-02,342.986522,0,Status Quo,Afghanistan,High-violence,High-violence
79,178.0,3,0.0,2,319.333333,412.847128,3,0.0,2,0.000045,0.040971,0.517609,0.392436,4.893867e-02,354.670346,0,Status Quo,Nigeria,High-violence,High-violence
60,215.0,3,1.0,1,49.333333,251.402323,3,0.0,2,0.000172,0.064751,0.592572,0.316076,2.642897e-02,108.757159,0,Status Quo,Iraq,High-violence,High-violence
120,110.0,3,1.0,2,118.666667,220.566761,3,0.0,2,0.000243,0.072443,0.608961,0.296072,2.228132e-02,279.134945,0,Status Quo,Somalia,High-violence,High-violence
149,102.0,3,0.0,2,161.666667,216.431120,3,0.0,2,0.000255,0.073608,0.611199,0.293205,2.173204e-02,217.040752,0,Status Quo,Myanmar,High-violence,High-violence
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
102,0.0,0,0.0,0,0.000000,0.021086,0,0.0,0,0.977719,0.017761,0.004472,0.000047,4.609274e-08,0.025520,0,Status Quo,Slovakia,Non-violent,Non-violent
231,0.0,0,0.0,0,0.000000,0.020954,0,0.0,0,0.977725,0.017757,0.004471,0.000047,4.606932e-08,0.017503,0,Status Quo,Montenegro,Non-violent,Non-violent
84,0.0,0,0.0,0,0.000000,0.020891,0,0.0,0,0.977727,0.017755,0.004470,0.000047,4.605816e-08,0.018482,0,Status Quo,Croatia,Non-violent,Non-violent


## FatCat Lists Condensed

This cell creates FatCat lists that are condensed to only include: 

    - Country Name
    - FatCat_change_nominal
    - Fatcat_0_nominal
    - Fatcat_12_nominal
    - Fat0
    - Fat12


In [30]:
deescalationlist_short = deescalationlist[['country', 'facat_change_nominal', 'fatcat_0_nominal', 'fatcat_12_nominal',
                                           'fatalities_step_0', 'fatalities_step_12','rolling_mean_violence_step_0', 'rolling_mean_violence_step_12']]



In [31]:
deescalationlist_short

,country,facat_change_nominal,fatcat_0_nominal,fatcat_12_nominal,fatalities_step_0,fatalities_step_12,rolling_mean_violence_step_0,rolling_mean_violence_step_12
country_id,,,,,,,,
117,Ukraine,De-escalation,Extreme Violence,Violent,4405.0,81.222141,6854.000000,128.723966
78,Niger,Low De-escalation,High-violence,Violent,117.0,56.799116,64.000000,53.243377
137,Tajikistan,Low De-escalation,Violent,Low Violence,46.0,1.343148,0.333333,1.446490


In [32]:
deescalationlist_short.rename(columns={'country': 'Country','facat_change_nominal': 'Estimated Change in Violence',
                                      'fatcat_0_nominal': 'Present Violence Level','fatcat_12_nominal': 'Predicted Violence Level',
                                      'fatalities_step_0': 'Present Fatalities','rolling_mean_violence_step_0': 'Mean Violence 3-months Prior (present)',
                                      'rolling_mean_violence_step_12': 'Mean Violence 3-months Prior (predicted)','fatalities_step_12': 'Predicted Fatalities'},
                              inplace = True)

In [33]:
deescalationlist_short
#print(deescalationlist_short.to_markdown())

,Country,Estimated Change in Violence,Present Violence Level,Predicted Violence Level,Present Fatalities,Predicted Fatalities,Mean Violence 3-months Prior (present),Mean Violence 3-months Prior (predicted)
country_id,,,,,,,,
117,Ukraine,De-escalation,Extreme Violence,Violent,4405.0,81.222141,6854.000000,128.723966
78,Niger,Low De-escalation,High-violence,Violent,117.0,56.799116,64.000000,53.243377
137,Tajikistan,Low De-escalation,Violent,Low Violence,46.0,1.343148,0.333333,1.446490


### De-escalation

In [34]:
escalationlist_short = escalationlist[['country', 'facat_change_nominal', 'fatcat_0_nominal', 'fatcat_12_nominal',
                                           'fatalities_step_0', 'fatalities_step_12','rolling_mean_violence_step_0', 'rolling_mean_violence_step_12']]


In [40]:
#remove countries with less than 1 fatality
escalationlist_short = escalationlist_short.query('fatalities_step_12 >=1')
escalationlist_short.sort_values('fatalities_step_12', ascending=False)

,country,facat_change_nominal,fatcat_0_nominal,fatcat_12_nominal,fatalities_step_0,fatalities_step_12,rolling_mean_violence_step_0,rolling_mean_violence_step_12
country_id,,,,,,,,
124,Yemen,Low Escalation,Violent,High-violence,41.0,581.768308,258.000000,464.244780
220,Syria,Low Escalation,Violent,High-violence,85.0,233.068575,59.000000,191.362203
223,India,Low Escalation,Violent,High-violence,38.0,116.224225,30.333333,99.473261
237,Kenya,Low Escalation,Non-violent,Low Violence,0.0,4.422479,9.666667,5.858108
96,Turkey,Low Escalation,Non-violent,Low Violence,0.0,3.655749,0.333333,3.657932
246,South Sudan,Low Escalation,Non-violent,Low Violence,0.0,3.394828,0.333333,3.762523
214,Chad,Low Escalation,Non-violent,Low Violence,0.0,3.236308,9.000000,3.979258
245,Sudan,Low Escalation,Non-violent,Low Violence,0.0,2.464283,0.000000,2.160593
128,Iran,Low Escalation,Non-violent,Low Violence,0.0,2.252714,0.000000,2.725371


# Descripitives 

These cells define descriptive statistics for FatCat

In [ ]:
# Print Column names

for col in fatcat_list.columns:
    print(col)

# To Do List:
	1. create a better fatcat. Maybe fatcat should be the mean level of violence for the 3 months prior plus the current month? No, probably not. 
	2. Better fatcat. If month X is has less deaths than mean 3 months prior, is that a good way to categorize FatCat?
	3. Can (and should) create a multi-nomial logit that gives the probability a state will transition between any of the conflict states
	4. The multi-nomial logit will provide the ability to assign risk to countries

## Introduction

This document outlines two tools VIEWS-users can engage with to employ and visualize the data: FatCat and FatCat Lists. FatCat is a VIEWS tool which categorized VIEWS data into fatalities categories. FatCat Lists, is a visualization tool which allows users to quickly generate lists of countries that are predicted to transition between states of violence. 

This document begins by introducing FatCat, explaining its purpose, defining the operationalization of its variables, and outlines its functions. Following the overview of FatCat, the document introduces FatCat Lists, and explains its functionality for users.  

<br />

# FatCat
## <u>Purpose</u>

FatCat is a tool that categorizes country-month (cm) violence into classes and presents these data in a comparison list. The function creates a pandas DataFrame that fetches conflict actuals for the 40 months prior to EndOfHistory (as report by the UCDP GED and UCDP candidate datasets) and VIEWS predictions from EndOfHistory+1 to EndOfHistory+36. 

This combined DataFrame indexes these data by country_id-step. Country_id refers to the country's assigned Gledtisch-Ward country code, whereas steps refer to the number of months away the observation is from the EndOfHistory. EndOfHistory is defined as the last month of available UCDP candidate data (*actuals*). 

Here, Step0 refers to the user defined EndOfHistory. The negative steps represent months since EndOfHistory (*actuals*) and positive steps correspond to steps forward since EndOfHistory (*predictions*). 

<br/>

##### Fatalities Categories

These data are then categorized by the number of fatalities at each country-step as follows:<br/>
**<u>Non-violent</u>**: The country-step has zero estimated conflict-related fatalities <br/>
**<u>Low Violence</u>**: The country-step has 1-9 estimated conflict-related fatalities<br/>
**<u>Violent</u>**: The country-step has 10-99 estimated conflict-related fatalities<br/>
**<u>High Violence</u>**: The country-step has 100-999 estimated conflict-related fatalities<br/>
**<u>Extreme Violence</u>**: The country-step has 1000+ estimated conflict-related fatalities<br/>

<br/>

##### Monthly Changes

Next, the 'monthly_change' variable calculates the change between levels of violence from one month to the next. For example, whether a country escalates or de-escalates from step(x) to step(y). These changes are categorized as: 

**<u>Extreme De-scalation</u>**: When a country changes  from extreme violence to non-violent in one month<br/>
**<u>Rapid De-escalation</u>**: When a country moves from high violence to non-violent in one month<br/>
**<u>De-Escalation</u>**: When a country moves from high violent to non-violent in one month<br/>
**<u>Low De-esclation</u>**: When a country moves from low violence to non-violent in one month<br/>
**<u>Staus Quo</u>**: When a country does not observe a change in violence level form month to month<br/>
**<u>Low Escalation</u>**: When a country moves from non-violent low violence to in one month<br/>
**<u>Escalation</u>**: When a country moves from non-violent to violent in one month<br/>
**<u>Rapid Escalation</u>**: When a country moves from non-violent to high violent in one month<br/>
**<u>Extreme Escalation</u>**: When a country moves from non-violent to high violence in one month<br/>

<br/>

##### Rolling Violence and Intensity Levels

After the creation of 'monthly change', the variables 'rolling_mean_violence' and 'intensity_lvl' are created. The variable 'rolling_mean_violence' takes the mean number of fatalities over the user defined months prior (currently set to three months prior). This variable provides context on the baseline level of recent violence. The 'intensity_lvl' variable is created by categorizing the 'rolling_mean_violence' into three categories: 
<br/>
**<u>0</u>**: The mean fatalities in the 3-months prior is 0 <br/>
**<u>1</u>**: The mean fatalities in the 3-months prior is less than 100<br/>
**<u>2</u>**: The mean fatalities in the 3-months prior is greater than 100<br/>

<br/>
<br/>

##### Uncertainty of Predictions

All positive steps (*predictions*) have four additional variables. These variables capture the probability that a given country-step prediction will fall into the corresponding categories:

<br/>
<u>multi_0_logit</u>: The probability that fatalities in the step have 0 fatalitites  <br />
<u>multi_1_logit</u>:  The probability that fatalities in the step have 1-9 fatalitites  <br />
<u>multi_2_logit</u>: The probability that fatalities in the step have 9-99 fatalitites  <br />
<u>multi_3_logit</u>: The probability that fatalities in the step have 100-999 fatalitites  <br />
<u>multi_4_logit</u>: The probability that fatalities in the step have 1000+ fatalitites  <br />

<br/>

## FatCat Lists

Whereas FatCat provides a Pandas DataFrame with relevant information on changes in violence levels for given countries, FatCat Lists provides a list of countries predicted to change from one fatcat status to another over a user-specified period. 
<br/>
For example, the photo below presents the list of countries that are expected to de-escalate in violence from EndOfHistory to EndOfHistory+12 months. This table presents the status of the country at present (EndOfHistory) and the predicted level of violence in the future (here one-year later). FatCat Lists also provides the number of fatalities at each point and the mean level of violence in the 3-months preceding the given month. 


FatCat Lists allows for users to produce lists of countries that move between any of the possible FatCats over any specified time period. 

<br/>
<br/>

## What FatCat does not do

1. FatCat provides the ability to compare change across two steps. (e.g., Step0 vs Step12). FatCat does not allow for the comparison of multiple steps, yet. <br/>
<br/>
2. This version of FatCat allows for the comparison of a future step against Step0; it does not allow for the comparison of future steps against another. For example, Step0 vs Step12 works. Step6 vs Step12 does not work, yet. <br/>


## Future Ambitions

A future version of FatCat uses these data to create variables which capture the probability that a given country will shift between any of the FatCats. To conduct this, these data will be modeled in a multinomial logit. This equation will produce the likelihood that a given step will shift between states of conflict, only taking into account its past conflicts. 

<br/>
With these results, FatCat can assign risk categories to country-steps based on the probability that the country will escalate in violence. These data can be used as future input data for models that seek to study how specific interventions alter the probability of violence de-/escalation. 

<br/>
<br/>


## FatCat Variable Descriptions

This section provides information on the variables defined in FatCat. 

<u>fatalities_step_0</u>: The number of fatalities at step 0 (integer) <br />
<u>fatcat_step_0</u>: The fatalities category at step0 (categorical) <br />
<u>fatcat_change_step_0</u>: The change in category from step -1 to step0 (integer) <br />
<u>rolling_mean_violence_step_0</u>: Mean number of fatalities in the 3-months prior to step 0.  (integer) <br />
<u>intensity_lvl_step_0</u>: Categorical variable defining the level of intensity in the three months prior to step0. (0 = no fatalities, 1 = 1-99 fatalities, 2 = more than 100 fatalities ) <br />

<u>fatalities_step_12</u>: The number of fatalities at step 12 (integer) <br />
<u>fatcat_step_12</u>: The fatalities category at step 12 (categorical) <br />
<u>fatcat_change_step_12</u>: The change in category from step 11 to step 12 (integer) <br />
<u>intensity_lvl_step_12</u>: Categorical variable defining the level of intensity in the three months prior to step 12. (0 = no fatalities, 1 = 1-99 fatalities, 2 = more than 100 fatalities ) <br />

<u>multi_0_logit</u>: The probability that fatalities from step 12 have 0 fatalitites  <br />
<u>multi_1_logit</u>: The probability that fatalities from step 12 have 1-9 fatalitites  <br />
<u>multi_2_logit</u>: The probability that fatalities from step 12 have 9-99 fatalitites  <br />
<u>multi_3_logit</u>: The probability that fatalities from step 12 have 100-999 fatalitites  <br />
<u>multi_4_logit</u>: The probability that fatalities from step 12 have 1000+ fatalitites  <br />

<u>rolling_mean_violence_step_12</u>:  Mean number of fatalities in the 3-months prior to step 12.  (integer) <br />
<u>fatcat_change</u>: The change in fatcat when moving from step 0 to step 12. (integer ranging from -4 to 4) <br />
<u>facat_change_nominal</u>: The nominal category for fatcat_change. (-4: extreme de-esclation; -3: high de-esclation; -2 de-esclation; -1: low de-escalation;  0: status quo; 1: low escalation; 2: esclation; 3: high escalation; 4: extreme escalation) <br />

<u>country</u>: Country name <br />
<u>fatcat_0_nominal</u>: Category of violence at step 0 ('Non-violent', 'Low Violence', 'Violent', 'High-violence', 'Extreme Violence') <br />
<u>fatcat_12_nominal</u>  (Category of violence at step 0 ('Non-violent', 'Low Violence', 'Violent', 'High-violence', 'Extreme Violence') <br />